In [40]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

import torch
import torch.nn as nn


from pathlib import Path
import os
from classes import *

In [41]:
loaddata = LoadData()


In [42]:

file = str(Path('FA_20240517_2H_yeast_Nicotinamide-d4 _9.csv'))
substrates = loaddata.get_substrate_list(file)
metabolites = loaddata.get_metabolite_list(file)
df_list = loaddata.load_data(file)

In [43]:
substrates, metabolites

([9.094, 8.876, 8.42, 7.772, np.float64(4.7)], [9.031, 8.714, 8.376, 7.659])

In [84]:
from scipy.ndimage import gaussian_filter1d


def fill_df(df):
    """If Data is not ranging from -2 to 10, fill the data with noise
    
    """

    def noise_func(y_new, noise):
        # smooth the noise with gaussian filter
        # Berechne die Standardabweichung der y-Daten
        sigma = len(y_new) / 100
        noise = gaussian_filter1d(input=noise, sigma=sigma)
        return noise

    df.rename(columns={df.columns[0]: 'x'}, inplace=True)

    x = df.iloc[:, 0]

    for i in range(1, len(df.columns)):
        y = df.iloc[:, i]



        # Calculate the step size of the x values
        x_diff = np.diff(x)
        step = np.mean(x_diff)
        
        # Take sample range for the noise
        x_range_lower = 20
        x_range_upper = 100

        # get sample data of the noise
        x_sample = x[x_range_lower:x_range_upper]
        y_sample = y[x_range_lower:x_range_upper]

        # get the max and min values of the sample data for the noise
        y_min, y_max = y_sample.min(), abs(y_sample.min())
        
        # get the max and min values of the x data
        x_min, x_max = x.min(), x.max()
        
        #print(f'x_xample: {x_sample}, y_sample: {y_sample}')
        #print(f'x_min: {x_min}, x_max: {x_max}')
        #print(f'y_min: {y_min}, y_max: {y_max}')

        if x_min > -2:

            # create values in n steps
            x_new = np.arange(-2, x.iloc[0], step)
            
            y_new = np.zeros_like(x_new)

            # set the noise level
            noise = np.random.normal(y_min, y_max, len(x_new))
            
            # smooth the noise with gaussian filter
            noise = noise_func(y_new, noise)

            # replace y_new with noise
            y_new = noise
            #print(y_new)

            #df.iloc[:, i] = y_new
            data = pd.DataFrame({'x': x_new, df.columns[i]: y_new})
            #display(data)
            df = pd.concat([data, df], axis=0)
            df.reset_index(drop=True, inplace=True)

        if x_max < 10:
            
            
            # create values in n steps
            x_new = np.arange(x.iloc[-1], 10, step)
            #print(x_new)
            y_new = np.zeros_like(x_new)

            # set the noise level
            noise = np.random.normal(y_min, y_max, len(x_new))

            # smooth the noise with gaussian filter
            noise = noise_func(y_new, noise)

            # replace y_new with noise
            y_new = noise

            data = pd.DataFrame({'x': x_new, df.columns[i]: y_new})
            df = pd.concat([df, data], axis=0)
            df.reset_index(drop=True, inplace=True)
            
        if i == 2:

            break
        
                
    return df

In [85]:
df = pd.read_csv(df_list[0])
#display(df)
df = fill_df(df)
display(df)

,x,FA_20240517_2H_yeast_1.9.ser#2,FA_20240517_2H_yeast_1.9.ser#1,FA_20240517_2H_yeast_1.9.ser#3,FA_20240517_2H_yeast_1.9.ser#4,FA_20240517_2H_yeast_1.9.ser#5,FA_20240517_2H_yeast_1.9.ser#6,FA_20240517_2H_yeast_1.9.ser#7,FA_20240517_2H_yeast_1.9.ser#8,FA_20240517_2H_yeast_1.9.ser#9,...,FA_20240517_2H_yeast_1.9.ser#41,FA_20240517_2H_yeast_1.9.ser#42,FA_20240517_2H_yeast_1.9.ser#43,FA_20240517_2H_yeast_1.9.ser#44,FA_20240517_2H_yeast_1.9.ser#45,FA_20240517_2H_yeast_1.9.ser#46,FA_20240517_2H_yeast_1.9.ser#47,FA_20240517_2H_yeast_1.9.ser#48,FA_20240517_2H_yeast_1.9.ser#49,FA_20240517_2H_yeast_1.9.ser#50
0,-2.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1.991855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-1.983710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-1.975565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-1.967420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2170,11.142700,438.474,-569.8060,516.165,-1437.25,2731.21,-535.8490,-334.935,-889.118,533.676,...,-251.224,633.567,-940.886,2255.99,359.295,-684.584,718.162,2176.330,-436.282,-2302.63
2171,11.150900,319.802,-260.3650,272.143,-1731.01,2960.16,-41.1904,-159.226,-976.832,-446.650,...,-385.964,332.352,-677.787,3004.31,932.641,-357.151,1296.970,2115.620,-214.366,-2286.14
2172,11.159000,176.833,-15.3604,-322.269,-2119.45,2755.96,327.6070,131.775,-919.698,-619.858,...,-348.141,255.558,-553.591,3148.39,1315.260,-446.132,1630.200,1937.050,173.100,-2172.04
2173,11.167200,145.307,63.6071,-1078.250,-2397.04,2750.27,382.3540,551.996,-780.292,-423.580,...,-442.490,472.343,-253.644,2721.18,1171.890,-585.413,1372.170,1359.150,736.266,-2033.80


In [46]:
def get_pictrue(df):
    pic = df.iloc[:, 1:]
    return pic

pic = get_pictrue(df)

print(pic.shape)

x = df.iloc[:, 0]
print(x.shape)

(28863, 50)
(28863,)


In [47]:
import numpy as np
from scipy.ndimage import zoom

# Original array
# Target shape
target_shape = (500, 50)


# Calculate scaling factors for each dimension
scale_factors = (target_shape[0] / pic.shape[0], target_shape[1] / pic.shape[1])

# Apply zoom with the calculated scale factors
upscaled_array = zoom(pic, scale_factors, order=3)


x_shape = (pic.shape[0], )
scale_factors_x = (target_shape[0] / x_shape[0], )
upscaled_x = zoom(x, scale_factors_x, order=3)


print(upscaled_array.shape)
print(upscaled_x.shape)


(500, 50)
(500,)


In [48]:
import plotly.express as px
y_labels = upscaled_x

def get_x_labels(column_list):
    x_labels = []
    for i in column_list:

        x_labels.append(i.split('#')[-1])
    return x_labels

x_labels = get_x_labels(df.columns[1:])

fig = px.imshow(upscaled_array, color_continuous_scale='Spectral_r', aspect='auto', 
                #y=y_labels, x=x_labels,
                )
fig.show()

In [49]:
print(upscaled_x)

[-2.00000000e+00 -1.52889418e+00 -1.05778836e+00 -5.86682533e-01
 -1.15576711e-01  3.55529111e-01  8.26634934e-01  1.29774076e+00
  1.76884658e+00  2.23995240e+00 -1.81757748e+00 -1.34647166e+00
 -8.75365839e-01 -4.04260017e-01  6.68458055e-02  5.37951628e-01
  1.00905745e+00  1.48016327e+00  1.95126909e+00  2.42237493e+00
 -1.63515497e+00 -1.16404914e+00 -6.92943323e-01 -2.21837500e-01
  2.49268322e-01  7.20374144e-01  1.19147997e+00  1.66258579e+00
  2.13369161e+00 -1.92383481e+00 -1.45273245e+00 -9.81626628e-01
 -5.10520806e-01 -3.94149839e-02  4.31690838e-01  9.02796661e-01
  1.37390248e+00  1.84500831e+00  2.31611413e+00 -1.74141576e+00
 -1.27030993e+00 -7.99204112e-01 -3.28098290e-01  1.43007533e-01
  6.14113355e-01  1.08521918e+00  1.55632500e+00  2.02743082e+00
  2.51997262e+00 -1.55899324e+00 -1.08788742e+00 -6.16781596e-01
 -1.45675773e-01  3.25430049e-01  7.96535871e-01  1.26764169e+00
  1.73874752e+00  2.20985334e+00 -1.84767655e+00 -1.37657072e+00
 -9.05464901e-01 -4.34359